The aim of this notebook is to explore unlabelled part of FLARE2022 dataset.

# Imports & constants

In [1]:
import sys 
sys.path.append('..')

In [2]:
import random
import plotly.express as px

from pathlib import Path
from types import SimpleNamespace

from monai.data import Dataset
from src.transforms import get_ssl_transforms

In [3]:
DATA_DIR = '/home/ditto/Downloads/FLARE22_UnlabeledCase1001-1500'  # A subsample of data

In [4]:
args = SimpleNamespace(**{
    'a_min': -500, 'a_max': 500, 
    'size_x': 1.0, 'size_y': 1.0, 'size_z': 2.5
})

# Exploration

In [5]:
img_paths = list(Path(DATA_DIR).glob('*.nii.gz'))
random.shuffle(img_paths)
data = [
    {'img': img_path} for img_path in img_paths[:100]
]

Let's check dims after preprocessing to make sure that cropping 96x96x96 crops
makes sense (it's not too much) and sampling positives with IoU=0 is faily easy
to sample.

In [7]:
transforms = get_ssl_transforms(args, mode='preprocess')  # Basic transforms only, no positive crops yet

The following transforms pipeline will be used: [<monai.transforms.io.dictionary.LoadImaged object at 0x7f4c63cd3490>, <monai.transforms.utility.dictionary.EnsureChannelFirstd object at 0x7f4c63cd3400>, <monai.transforms.spatial.dictionary.Orientationd object at 0x7f4ccc6a7e80>, <monai.transforms.spatial.dictionary.Spacingd object at 0x7f4c63b3d240>, <monai.transforms.intensity.dictionary.ScaleIntensityRanged object at 0x7f4c63b3d060>, <monai.transforms.croppad.dictionary.CropForegroundd object at 0x7f4c63b3d1b0>, <monai.transforms.croppad.dictionary.SpatialPadd object at 0x7f4c63b3d1e0>, <monai.transforms.utility.dictionary.EnsureTyped object at 0x7f4c63b3d450>].


/home/ditto/envs/sslct/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


In [8]:
dataset = Dataset(data, transform=transforms)

In [11]:
xs, ys, zs = [], [], []
for i in range(len(dataset)):
    dims = dataset[i]['img'].shape[1:]  # Skip batch dim
    ys.append(dims[0])
    xs.append(dims[1])
    zs.append(dims[2])

In [15]:
fig = px.scatter_3d(x=xs, y=ys, z=zs, width=800, height=800)
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()

In [16]:
min(xs), max(xs)

(228, 415)

In [17]:
min(ys), max(ys)

(300, 492)

In [18]:
min(zs), max(zs)

(96, 175)

The smallest CT for this subset is 308x228x96. If cropping 96x96x96 patches with max IoU = 0, it might be the case
that if the first one was cropped around the center, then the second one would be impossible (or almost impossible)
to crop (IoU would be always greater than 0). So the IoU function has to be modified a bit so that the first crop is sampled multiple times as well.